
##STEP 1: Import Libraries

In [3]:
import os
import torchvision
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader
from torch import nn
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.nn import Linear

import torch.nn as nn
import torch.optim as optim

##STEP 2: Data Preparations

In [4]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
from google.colab import files

#uploading files
upload_train = files.upload()

Saving cats&dogsfiltered.zip to cats&dogsfiltered.zip


In [11]:
import zipfile
import os

In [12]:

# Define the name of the uploaded zip file
zip_file_name = list(upload_train.keys())[0]

In [13]:
# Unzip the file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('/content/images')

# List the extracted files
extracted_files = os.listdir('/content/images')
print(extracted_files)

['cats_and_dogs_filtered']


##loading datasets

In [14]:
train_dataset = datasets.ImageFolder('/content/images/cats_and_dogs_filtered/train', transform=transform)
valid_dataset = datasets.ImageFolder('/content/images/cats_and_dogs_filtered/validation', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


##Step 4: Define the Model
Using of a pre-trained model (ResNet18) and to fine-tune it.

In [15]:
model = models.resnet18(pretrained=True)


# Modify the final layer to match the number of classes (dogs and cats)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)


# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:03<00:00, 14.0MB/s]


In [16]:
device

device(type='cuda')

#Step 5: Define Loss Function and Optimizer

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Step 6: Training the Model

In [18]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc

def validate(model, valid_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(valid_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc

num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    valid_loss, valid_acc = validate(model, valid_loader, criterion, device)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.2f}%')


Epoch 1/10, Train Loss: 0.2987, Train Acc: 88.35%, Valid Loss: 0.2170, Valid Acc: 91.00%
Epoch 2/10, Train Loss: 0.1214, Train Acc: 95.35%, Valid Loss: 0.3358, Valid Acc: 87.70%
Epoch 3/10, Train Loss: 0.1528, Train Acc: 93.60%, Valid Loss: 0.1500, Valid Acc: 93.50%
Epoch 4/10, Train Loss: 0.0871, Train Acc: 96.60%, Valid Loss: 0.1206, Valid Acc: 94.90%
Epoch 5/10, Train Loss: 0.0315, Train Acc: 98.75%, Valid Loss: 0.1357, Valid Acc: 94.50%
Epoch 6/10, Train Loss: 0.1019, Train Acc: 96.60%, Valid Loss: 0.2648, Valid Acc: 91.20%
Epoch 7/10, Train Loss: 0.0953, Train Acc: 96.30%, Valid Loss: 0.1656, Valid Acc: 93.40%
Epoch 8/10, Train Loss: 0.0472, Train Acc: 98.40%, Valid Loss: 0.1700, Valid Acc: 94.20%
Epoch 9/10, Train Loss: 0.0711, Train Acc: 97.40%, Valid Loss: 0.2172, Valid Acc: 92.70%
Epoch 10/10, Train Loss: 0.0301, Train Acc: 98.75%, Valid Loss: 0.1855, Valid Acc: 94.40%


#Step 7: Save the Model

In [20]:
torch.save(model.state_dict(), 'modelDvsC.pth')


#Step 8: Load and Test the Model

In [28]:
from PIL import Image
import matplotlib.pyplot as plt

model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.load_state_dict(torch.load('modelDvsC.pth'))
model = model.to(device)

# Function to test the model on a single image
def predict_image(image_path, model, device):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = outputs.max(1)

    return predicted.item()

image_path = 'download.jpeg'
predicted_class = predict_image(image_path, model, device)
print(f'Prediction: {"Dog" if predicted_class == 1 else "Cat"}')
print(f'Predicted Class: {predicted_class}')


Prediction: Dog
Predicted Class: 1
